In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from datetime import datetime

In [2]:
##/home/sherief/dev/WesternPower/LondonSmartMeter
DATA_ROOT = "smart-meters-in-london"
BLOCK_PATH = os.path.join(DATA_ROOT,'halfhourly_dataset') 
BLOCKS = os.listdir(BLOCK_PATH)

In [3]:
df = pd.concat([pd.read_csv(os.path.join(BLOCK_PATH,block)) for block in BLOCKS[:2]])
df['time']=  df['tstp'].astype(str).str[:19]

In [4]:
hourly_weather = pd.read_csv(os.path.join(DATA_ROOT, "weather_hourly_darksky.csv"))
daily_weather  = pd.read_csv(os.path.join(DATA_ROOT, "weather_daily_darksky.csv"))
house          = pd.read_csv(os.path.join(DATA_ROOT, "informations_households.csv"))
holidays       = pd.read_csv(os.path.join(DATA_ROOT, "uk_bank_holidays.csv"))

In [5]:
hourly_weather.columns = ["h_"+ c for c in hourly_weather.columns]
hourly_weather.rename(columns={"h_time":"time"}, inplace=True)

In [6]:
df = pd.merge(df, house, on='LCLid', how='left')

In [7]:
df = pd.merge(df, hourly_weather, on='time',  how='left')

In [8]:
df['month']=[ int(v[5:7]) for v in df['tstp'].values]
df['dayofweek']=[datetime.strptime(v[:-6], '%Y-%m-%d %H:%M:%S.%f').weekday()  for v in df['tstp'].values]

In [9]:
hols = holidays['Bank holidays'].values
df['isHoliday']=[v[:10] in hols  for v in df['tstp'].values]

In [10]:
def half_hour_of_day(datestring, formatstr='%Y-%m-%d %H:%M:%S.%f', stripChars=6):
    if stripChars>0:
        tmp = datetime.strptime(datestring[:-stripChars], formatstr)
    else:
        tmp = datetime.strptime(datestring, formatstr)
    return 2*(tmp.hour + (tmp.minute /60.0))

In [11]:
def minute_of_day(datestring, formatstr='%Y-%m-%d %H:%M:%S.%f', stripChars=6):
    if type(datestring)==float:
        return np.nan
    if stripChars>0:
        tmp = datetime.strptime(datestring[:-stripChars], formatstr)
    else:
        tmp = datetime.strptime(datestring, formatstr)
    return ((tmp.hour*60) + tmp.minute )

In [12]:
def get_ts_int(datestring, formatstr='%Y-%m-%d %H:%M:%S.%f', stripChars=6):
    if type(datestring)==float:
        return np.nan
    if stripChars>0:
        tmp = datetime.strptime(datestring[:-stripChars], formatstr)
    else:
        tmp = datetime.strptime(datestring, formatstr)
    start = datetime(2000,1, 1 )
    delta = tmp-start
    return delta.days*24*3600 + delta.seconds

In [13]:
df['halfhourofday'] = df.tstp.apply(lambda x: half_hour_of_day(x) )
df['date']=[ v[:10] for v in df['tstp'].values]  

In [14]:
df['ts_int'] = df.tstp.apply(lambda x: get_ts_int(x))

In [15]:
daily_weather['month']=[ int(v[5:7]) for v in daily_weather['time'].values]
def convert_time_cols(data_frame, column_name):
    data_frame[column_name+ "_mod"] = data_frame[column_name].apply(lambda x: minute_of_day(x, formatstr='%Y-%m-%d %H:%M:%S', stripChars=0) )

for c in ['temperatureMinTime', 'temperatureMaxTime', 'apparentTemperatureMinTime','apparentTemperatureMaxTime','temperatureHighTime','temperatureLowTime','apparentTemperatureHighTime','apparentTemperatureLowTime','sunsetTime','sunriseTime','uvIndexTime']:
    convert_time_cols(daily_weather, c)
daily_weather['date']=[ v[:10] for v in daily_weather['temperatureMinTime'].values]  
daily_weather.fillna(method='ffill', inplace=True)    

In [16]:
df = pd.merge(df, daily_weather, on='date', how='left')

In [17]:
df.sort_values(by='ts_int', ascending=True, inplace=True)

In [18]:
#fill the missing half hour rows
hcols =['h_visibility', 'h_windBearing', 'h_temperature','h_dewPoint', 'h_pressure', 'h_apparentTemperature', 'h_windSpeed',
       'h_precipType', 'h_icon', 'h_humidity', 'h_summary']
df[hcols]= df[hcols].ffill()

#run this to display a row from the main df 
for c in  df.columns:
    print(str(df.loc[3][c]) +  "____________________________" + c)  

In [20]:
label_col = 'energy(kWh/hh)'
categorical_cols = ['LCLid', 'stdorToU', 'Acorn_grouped', 'Acorn', 'summary', 'h_summary', 'icon', 'h_icon', 'h_precipType']
numeric_cols = ['h_visibility','h_windBearing','h_temperature','h_dewPoint','h_pressure','h_apparentTemperature'
,'h_windSpeed','h_humidity','month_x','dayofweek','isHoliday','halfhourofday','temperatureMax'
,'windBearing','dewPoint','cloudCover','windSpeed','pressure','apparentTemperatureHigh','visibility','humidity','apparentTemperatureLow'
,'apparentTemperatureMax','uvIndex','temperatureLow','temperatureMin','temperatureHigh','apparentTemperatureMin'
,'moonPhase','temperatureMinTime_mod','temperatureMaxTime_mod','apparentTemperatureMinTime_mod','apparentTemperatureMaxTime_mod'
,'temperatureHighTime_mod','temperatureLowTime_mod','apparentTemperatureHighTime_mod','apparentTemperatureLowTime_mod'
,'sunsetTime_mod','sunriseTime_mod','uvIndexTime_mod']


In [22]:
for c in categorical_cols:
    df[c] = df[c].astype("category").cat.codes

In [33]:
allcols = np.concatenate([categorical_cols ,  numeric_cols ]).tolist()
allcols.append(label_col)

In [36]:
df[allcols].to_csv('nn_input.csv', index=False)